In [1]:
import csv
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
# Read data from file
gps_data_path = "/mnt/hdd02/ba_copter_videos/Flight_05/DJIFlightRecord_2024-04-12_[18-34-58].csv"
vo_data_path = "/home/bastian/uni/visual_odometry_playground/orb_slam3/orb_slam3_ws/CameraTrajectory.csv"

# Write in file
gps_sync_path = '/mnt/hdd02/ba_copter_videos/Flight_05/gps_trajectory.traj'
vo_sync_path = '/mnt/hdd02/ba_copter_videos/Flight_05/vo_trajectory.traj'

In [3]:
def compute_quaternion_components(data):
    # Extracting data
    roll = np.deg2rad(data["OSD.roll"])
    pitch = np.deg2rad(data["OSD.pitch"])
    yaw = np.deg2rad(data["OSD.yaw"])

    # Initialize lists to store computed quaternion components
    x_list = []
    y_list = []
    z_list = []
    w_list = []

    # Compute quaternion components
    for r, p, y in zip(roll, pitch, yaw):
        r_obj = R.from_euler('xyz', [r, p, y], degrees=False)
        quaternion = r_obj.as_quat()
        x_list.append(quaternion[0])
        y_list.append(quaternion[1])
        z_list.append(quaternion[2])
        w_list.append(quaternion[3])
        
    return x_list, y_list, z_list, w_list

In [4]:
def create_gps_data_list(data):
    # Extracting data
    time = data["OSD.flyTime [s]"]
    pos_x = data["OSD.latitude"]
    pos_y = data["OSD.longitude"]
    pos_z_ft = data["OSD.height [ft]"] 
    pos_z_m = [ft * 0.3048 for ft in pos_z_ft] 
    quaternion_components = compute_quaternion_components(data)
    quaternion_x = quaternion_components[0]
    quaternion_y = quaternion_components[1]
    quaternion_z = quaternion_components[2]
    quaternion_w = quaternion_components[3]

    # Initialize list to store computed data
    data_list = []

    # Combine data into a list of tuples
    for t, x, y, z, qx, qy, qz, qw in zip(time, pos_x, pos_y, pos_z_m, quaternion_x, quaternion_y, quaternion_z, quaternion_w):
        # Append computed data to list
        data_list.append((t, x, y, z, qx, qy, qz, qw))
    
    return data_list

In [5]:
def create_vo_data_list(data):
    # Lists for delta values
    time = []
    cam_delta_twb_x = []
    cam_delta_twb_y = []
    cam_delta_twb_z = []
    quaternion_x = []
    quaternion_y = []
    quaternion_z = []
    quaternion_w = []

    # Read trajectory data and store delta values in lists
    with open(data, 'r') as f:
        for line in f:
            parts = line.split()
            time.append(float(parts[0]))
            cam_delta_twb_x.append(float(parts[1]))
            cam_delta_twb_y.append(float(parts[2]))
            cam_delta_twb_z.append(float(parts[3]))
            quaternion_x.append(float(parts[4]))
            quaternion_y.append(float(parts[5]))
            quaternion_z.append(float(parts[6]))
            quaternion_w.append(float(parts[7]))

    data_list = []

    for t, x, y, z, qx, qy, qz, qw in zip(time, cam_delta_twb_x, cam_delta_twb_y, cam_delta_twb_z, quaternion_x, quaternion_y, quaternion_z, quaternion_w):
        data_list.append((t, x, y, z, qx, qy, qz, qw))
    
    return data_list

In [6]:
def sync_times(gps_data, vo_data):
    # Extract timestamps from the datasets
    gps_times = [gps_point[0] for gps_point in gps_data]
    vo_times = [vo_point[0] / 1e9 for vo_point in vo_data]  # Conversion from nanoseconds to seconds

    # Set the start time to the minimum of both datasets
    gps_start_time = min(gps_times)
    vo_start_time = min(vo_times)

    # Synchronize the timestamps of both datasets
    synced_gps_times = [round(t - gps_start_time, 3) for t in gps_times]
    synced_vo_times = [round(t - vo_start_time, 3) for t in vo_times]

    # Synchronize GPS data
    synced_gps_data = []
    for gps_point, synced_time in zip(gps_data, synced_gps_times):
        synced_gps_data.append((synced_time, *gps_point[1:]))

    # Synchronize VO data
    synced_vo_data = []
    for vo_point, synced_time in zip(vo_data, synced_vo_times):
        synced_vo_data.append((synced_time, *vo_point[1:]))
    
    return synced_gps_data, synced_vo_data

In [7]:
def sync_gps_with_vo(gps_data, vo_data):
    synced_gps_data = []
    synced_vo_averages = []

    scale = 0.01
    lat= gps_data[0][1]
    lon = gps_data[0][2]
    height = gps_data[0][3]

    # Iterate through the GPS data
    for i in range(len(gps_data)):
        gps_time, gps_x, gps_y, gps_z, *gps_quaternion = gps_data[i]

        # Find the VO data points between the current and previous GPS data
        vo_points_between_gps = [vo_point for vo_point in vo_data if gps_data[i-1][0] < vo_point[0] < gps_time]

        if vo_points_between_gps:
            # Compute the average of the VO data
            vo_average_x = sum([vo_point[1] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_y = sum([vo_point[2] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_z = sum([vo_point[3] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_quaternion_x = sum([vo_point[4] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_quaternion_y = sum([vo_point[5] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_quaternion_z = sum([vo_point[6] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)
            vo_average_quaternion_w = sum([vo_point[7] for vo_point in vo_points_between_gps]) / len(vo_points_between_gps)

            # scale and average sync to coorinates
            vo_average_x = (vo_average_x * scale) + lat
            vo_average_y = (vo_average_y * scale) + lon
            vo_average_z = (vo_average_z * scale) + height
            
            # Append the synchronized data
            synced_gps_data.append((gps_time, gps_x, gps_y, gps_z, *gps_quaternion))
            synced_vo_averages.append((gps_time, vo_average_x, vo_average_y, vo_average_z,
                                        vo_average_quaternion_x, vo_average_quaternion_y,
                                        vo_average_quaternion_z, vo_average_quaternion_w))

    return synced_gps_data, synced_vo_averages


In [8]:
gps_data = create_gps_data_list(pd.read_csv(gps_data_path))
vo_data = create_vo_data_list(vo_data_path)

# sync time
time_synced_gps_data, time_synced_vo_data = sync_times(gps_data, vo_data)
# value sync
gps_data, vo_data = sync_gps_with_vo(time_synced_gps_data, time_synced_vo_data)

# Write data to file
with open(gps_sync_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(gps_data)

with open(vo_sync_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(vo_data)

print("Successful!")


Successful!
